# Tile Generation Tutorial

Welcome to the tile generation tutorial!

As a whole slide image is too large for deep learning model training, a slide is often divded into a set of small tiles, and used for training. For tile-based whole slide image analysis, generating tiles and labels is an important and laborious step. With LUNA tiling CLIs and tutorials, you can easily generate tile labels and get your data ready for downstream analysis. In this notebook, we will see how to generate tiles and labels using LUNA tiling CLIs. Here are the main steps we will review:

1. Load slides
2. Generate tiles, labels
3. Collect tiles for model training

Through out this notebook, we will use different method parameter files. Please refer to the example parameter files in the `configs` directory to follow these steps.


In [13]:
env DATASET_URL=file:///home/aukermaa/vmount/PRO_12-123/tables

env: DATASET_URL=file:///home/aukermaa/vmount/PRO_12-123/tables


This is the main tiling step. The CLI generates tiles, populates otsu and purple scores along with the regional annotation label. An otsu score is calculated using the otsu foreground/background detection algorithm commonly used to filter out the background of the slide. Purple scores are calculated to provide additional guidance to H&E slide analysis.


In [16]:
!generate_tiles \
/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
--tile_size 128 --requested_magnification 20 \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles


2022-04-20 19:42:48,591 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 19:42:50,336 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7fbef65faf70>
2022-04-20 19:42:50,338 - INFO - luna.common.utils - Validating params...
2022-04-20 19:42:50,340 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-04-20 19:42:50,341 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles
2022-04-20 19:42:50,342 - INFO - luna.common.utils -  -> Set tile_size (<class 'int'>) = 128
2022-04-20 19:42:50,343 - INFO - luna.common.utils -  -> Set requested_magnification (<class 'float'>) = 20.0
2022-04-20 19:42:50,346 - INFO - luna.common.utils - Expanding inputs...
2022-04-20 19:42:50,347 - INFO - luna.common.utils - Attempting to read metadata at /home/aukermaa/vmount

In [17]:
!label_tiles \
../PRO_12-123/data/toy_data_set/table/ANNOTATIONS /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/label

2022-04-20 19:42:53,281 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 19:42:53,282 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tile_labels at 0x7ff0d14889d0>
2022-04-20 19:42:53,284 - INFO - luna.common.utils - Validating params...
2022-04-20 19:42:53,285 - INFO - luna.common.utils -  -> Set input_slide_annotation_dataset (<class 'str'>) = ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS
2022-04-20 19:42:53,287 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles
2022-04-20 19:42:53,288 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/label
2022-04-20 19:42:53,290 - INFO - luna.common.utils - Expanding inputs...
2022-04-20 19:42:53,291 - INFO - luna.common.utils - Attempting to read metadata at ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS/metadata.yml
2022-04-20 19:42:53,295 - INFO - luna.

In [19]:
!save_tiles \
/home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs \
/home/aukermaa/vmount/PRO_12-123/tiling/test/tiles \
--num_cores 16 --batch_size 200 --dataset_id PRO_TILES \
-o /home/aukermaa/vmount/PRO_12-123/tiling/test/saved_tiles

2022-04-20 19:43:08,648 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 19:43:10,411 - INFO - luna.common.utils - Started CLI Runner wtih <function save_tiles at 0x7f0302a27af0>
2022-04-20 19:43:10,413 - INFO - luna.common.utils - Validating params...
2022-04-20 19:43:10,414 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-04-20 19:43:10,416 - INFO - luna.common.utils -  -> Set input_slide_tiles (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/tiles
2022-04-20 19:43:10,417 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/test/saved_tiles
2022-04-20 19:43:10,419 - INFO - luna.common.utils -  -> Set num_cores (<class 'int'>) = 16
2022-04-20 19:43:10,420 - INFO - luna.common.utils -  -> Set batch_size (<class 'int'>) = 200
2022-04-20 19:43:10,423 - INFO - luna.common.

In [66]:
from pathlib import Path
import subprocess
import os

class CLIShepard:
    def __init__(self, base_dir, uuid):
        self.base_dir = Path(base_dir).expanduser()
        self.uuid = uuid
        self.cli_steps = {}
    def run(self, cli_resource, *args, **kwargs):
        
        cli_name, cli_step = cli_resource.split('::')
        
        output_dir = os.path.join(self.base_dir, self.uuid, cli_step)

        if cli_name == 'bootstrap':
            self.cli_steps[cli_step] = Path(args[0]).expanduser()
            return None
        
        cli_call = [cli_name]
        for arg in args: 
            if arg in self.cli_steps.keys():
                cli_call.append(self.cli_steps[arg])
            else:
                cli_call.append(Path(arg).expanduser())
            
        for key, value in kwargs.items(): 
            cli_call.append(f"--{key}")
            cli_call.append(f"{value}")
            
        cli_call.append("-o")
        cli_call.append(output_dir)
        
        print (" ".join(f"{x}" for x in cli_call))
        
        subprocess.Popen(cli_call).communicate()
        
        self.cli_steps[cli_step] = output_dir

         
            
def pipeline (slide_id, input_slide, input_annotations):
    shepard = CLIShepard("~/vmount/PRO_12-123/tiling", slide_id)

    shepard.run(
        "bootstrap::slide", 
        input_slide, 
    )

    shepard.run(
        "bootstrap::annotations", 
        input_annotations, 
    )


    shepard.run(
        "generate_tiles::tiles", "slide", 
        tile_size=128, 
        requested_magnification=20
    )

    shepard.run(
        "label_tiles::labels", "annotations", "tiles",
    )

    shepard.run(
        "save_tiles::saved_tiles", "slide", "labels",
        num_cores=16, batch_size=200, dataset_id='PRO_TILES'
    )

In [67]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(5) as pool:
    pool.submit(pipeline, "01OV002-bd8cdc70-3d46-40ae-99c4-90ef77", "../PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs", "../PRO_12-123/data/toy_data_set/table/ANNOTATIONS")
    pool.submit(pipeline, "01OV002-ed65cf94-8bc6-492b-9149-adc16f", "../PRO_12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs", "../PRO_12-123/data/toy_data_set/table/ANNOTATIONS")
    pool.submit(pipeline, "01OV007-9b90eb78-2f50-4aeb-b010-d642f9", "../PRO_12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs", "../PRO_12-123/data/toy_data_set/table/ANNOTATIONS")
    pool.submit(pipeline, "01OV008-308ad404-7079-4ff8-8232-12ee2e", "../PRO_12-123/data/toy_data_set/01OV008-308ad404-7079-4ff8-8232-12ee2e.svs", "../PRO_12-123/data/toy_data_set/table/ANNOTATIONS")
    pool.submit(pipeline, "01OV008-7579323e-2fae-43a9-b00f-a15c28", "../PRO_12-123/data/toy_data_set/01OV008-7579323e-2fae-43a9-b00f-a15c28.svs", "../PRO_12-123/data/toy_data_set/table/ANNOTATIONS")



generate_tiles ../PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/tiles
generate_tiles ../PRO_12-123/data/toy_data_set/01OV002-ed65cf94-8bc6-492b-9149-adc16f.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-ed65cf94-8bc6-492b-9149-adc16f/tiles
generate_tiles ../PRO_12-123/data/toy_data_set/01OV007-9b90eb78-2f50-4aeb-b010-d642f9.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/tiles
generate_tiles ../PRO_12-123/data/toy_data_set/01OV008-308ad404-7079-4ff8-8232-12ee2e.svs --tile_size 128 --requested_magnification 20 -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-308ad404-7079-4ff8-8232-12ee2e/tiles
generate_tiles ../PRO_12-123/data/toy_data_set/01OV008-7579323e-2fae-43a9-b00f-a15c28.svs --tile_size 128 --

2022-04-20 20:19:49,748 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 20:19:49,766 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 20:19:49,814 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 20:19:49,835 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 20:19:50,022 - INFO - root - Initalized logger, log file at: data-processing.log
2022-04-20 20:19:51,808 - INFO - luna.common.utils - Started CLI Runner wtih <function generate_tiles at 0x7f67f7220f70>
2022-04-20 20:19:51,809 - INFO - luna.common.utils - Validating params...
2022-04-20 20:19:51,810 - INFO - luna.common.utils -  -> Set input_slide_image (<class 'str'>) = ../PRO_12-123/data/toy_data_set/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77.svs
2022-04-20 20:19:51,811 - INFO - luna.common.utils -  -> Set output_dir (<class 'str'>) = /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/tile

2022-04-20 20:19:52,016 - INFO - generate_tiles - Number of tiles in raster: 99258
2022-04-20 20:19:52,022 - INFO - generate_tiles - Slide size = [52800,38628]
2022-04-20 20:19:52,023 - INFO - luna.common.utils - Code block 'transform::generate_tiles' took: 0.20122752338647842s
2022-04-20 20:19:52,024 - INFO - generate_tiles - Normalized magnification scale factor for 20.0x is 2.0
2022-04-20 20:19:52,025 - INFO - generate_tiles - Requested tile size=128, tile size at full magnficiation=256
2022-04-20 20:19:52,025 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-20 20:19:52,025 - INFO - luna.common.utils -  Done with transform, running post-transform functions... 
2022-04-20 20:19:52,025 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-20 20:19:52,026 - INFO - generate_tiles - tiles x 207, tiles y 151
2022-04-20 20:19:52,030 - INFO - luna.common.utils - Done.
2022-04-20 20:19:52,038 - INFO 

2022-04-20 20:19:52,231 - INFO - generate_tiles -                  x_coord  y_coord  xy_extent  tile_size tile_units
2022-04-20 20:19:52,231 - INFO - generate_tiles - address                                                           
2022-04-20 20:19:52,231 - INFO - generate_tiles - x1_y1_z20.0          256      256        256        128         px
2022-04-20 20:19:52,231 - INFO - generate_tiles - x1_y2_z20.0          256      512        256        128         px
2022-04-20 20:19:52,231 - INFO - generate_tiles - x1_y3_z20.0          256      768        256        128         px
2022-04-20 20:19:52,231 - INFO - generate_tiles - x1_y4_z20.0          256     1024        256        128         px
2022-04-20 20:19:52,231 - INFO - generate_tiles - x1_y5_z20.0          256     1280        256        128         px
2022-04-20 20:19:52,231 - INFO - generate_tiles - ...                  ...      ...        ...        ...        ...
2022-04-20 20:19:52,231 - INFO - generate_tiles - x163_y226_z20.

label_tiles ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/tiles -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-bd8cdc70-3d46-40ae-99c4-90ef77/labels
label_tiles ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-308ad404-7079-4ff8-8232-12ee2e/tiles -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-308ad404-7079-4ff8-8232-12ee2e/labels
label_tiles ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-ed65cf94-8bc6-492b-9149-adc16f/tiles -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV002-ed65cf94-8bc6-492b-9149-adc16f/labels
label_tiles ../PRO_12-123/data/toy_data_set/table/ANNOTATIONS /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/tiles -o /home/aukermaa/vmount/PRO_12-123/tiling/01OV007-9b90eb78-2f50-4aeb-b010-d642f9/labels
label_tiles ../PRO_12-123/data/toy_data_set/table/ANNOTA

...

2022-04-20 20:20:28,732 - INFO - luna.common.utils - Code block 'transform::save_tiles' took: 24.42469407990575s
2022-04-20 20:20:28,750 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-20 20:20:28,750 - INFO - luna.common.utils -  Done with transform, running post-transform functions... 
2022-04-20 20:20:28,750 - INFO - luna.common.utils - ------------------------------------------------------------
2022-04-20 20:20:28,757 - INFO - luna.common.utils - Adding feature segment /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-7579323e-2fae-43a9-b00f-a15c28/saved_tiles/01OV008-7579323e-2fae-43a9-b00f-a15c28.tiles.parquet to PRO_TILES
2022-04-20 20:20:28,757 - INFO - luna.common.utils - Found dataset URL = file:///home/aukermaa/vmount/PRO_12-123/tables
2022-04-20 20:20:28,762 - INFO - luna.common.utils - Adding /home/aukermaa/vmount/PRO_12-123/tiling/01OV008-7579323e-2fae-43a9-b00f-a15c28/saved_tiles/01OV008-7579323e-2fae-43a9-b00f-a15c28.tile

 13%|█▎        | 66/497 [00:08<00:42, 10.12it/s]]

2022-04-20 20:20:34,726 - INFO - generate_tiles -                  x_coord  ...                                         tile_store
2022-04-20 20:20:34,726 - INFO - generate_tiles - address                   ...                                                   
2022-04-20 20:20:34,726 - INFO - generate_tiles - x1_y1_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:20:34,726 - INFO - generate_tiles - x1_y2_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:20:34,726 - INFO - generate_tiles - x1_y3_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:20:34,726 - INFO - generate_tiles - x1_y4_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:20:34,726 - INFO - generate_tiles - x1_y5_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:20:34,726 - INFO - generate_tiles - ...                  ...  ...    


100%|██████████| 497/497 [00:37<00:00, 13.14it/s]

2022-04-20 20:21:04,776 - INFO - generate_tiles -                  x_coord  ...                                         tile_store
2022-04-20 20:21:04,776 - INFO - generate_tiles - address                   ...                                                   
2022-04-20 20:21:04,776 - INFO - generate_tiles - x1_y1_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:21:04,776 - INFO - generate_tiles - x1_y2_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:21:04,776 - INFO - generate_tiles - x1_y3_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:21:04,776 - INFO - generate_tiles - x1_y4_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:21:04,776 - INFO - generate_tiles - x1_y5_z20.0          256  ...  /home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
2022-04-20 20:21:04,776 - INFO - generate_tiles - ...                  ...  ...    

In [76]:
pd.read_parquet("/home/aukermaa/vmount/PRO_12-123/tables/datasets/PRO_TILES/").query("intersection_area > 0")

,x_coord,y_coord,xy_extent,tile_size,tile_units,regional_label,intersection_area,tile_store
address,,,,,,,,
x52_y114_z20.0,13312,29184,256,128,px,stroma,0.055345,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
x53_y113_z20.0,13568,28928,256,128,px,stroma,0.212376,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
x53_y114_z20.0,13568,29184,256,128,px,stroma,0.965976,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
x53_y115_z20.0,13568,29440,256,128,px,stroma,0.344496,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
x54_y112_z20.0,13824,28672,256,128,px,stroma,0.061422,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
...,...,...,...,...,...,...,...,...
x111_y80_z20.0,28416,20480,256,128,px,stroma,1.000000,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
x111_y81_z20.0,28416,20736,256,128,px,stroma,0.577473,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...
x112_y79_z20.0,28672,20224,256,128,px,stroma,0.000450,/home/aukermaa/vmount/PRO_12-123/tiling/01OV00...


Congratulations! Now you have the tiles images and labels ready to train your model.